## **Exercise 1**

**Assuming you have a Brick sized (40mm, 20mm, 15mm) and we make a translation and rotation in function of Mobile frame,**

<img src="images/ex1.png" width="300"/>

**a) Find the Pose of Bframe respect World if pose of Aframe is (30,40,50,0,0,0)**

**b) Find the position of the center of gravity of brick with respect to the Bframe, before and after processing the movement**

**c) Find the position of the center of gravity of brick with respect to the A-frame, before and after processing the movement**

**d) Solve also the three previous questions analytically and using roboDK**


We will first open Exercises1_2.rdk

In [4]:
from pathlib import Path
import subprocess
import time
from robodk.robolink import Robolink, ITEM_TYPE_STATION

# === 1. Paths ===
notebook_dir = Path().resolve()
exe_path = Path(r"C:\RoboDK\bin\RoboDK.exe")
project_path = (notebook_dir / "Exercises1_2.rdk").resolve()

# === 2. Kill previous RoboDK instances ===
print("🔁 Closing any existing RoboDK instances...")
subprocess.call("taskkill /f /im RoboDK.exe", shell=True)
time.sleep(2)

# === 3. Start RoboDK with project as argument ===
print(f"🚀 Launching RoboDK with project: {project_path.name}")
subprocess.Popen([str(exe_path), str(project_path)])
time.sleep(2)

# === 4. Wait for API connection ===
print("⏳ Waiting for RoboDK to be ready...", end="")
connected = False
for _ in range(20):
    try:
        RDK = Robolink('localhost', 20500)
        station = RDK.Item('', ITEM_TYPE_STATION)
        if station.Valid():
            connected = True
            break
    except:
        pass
    time.sleep(0.5)
    print(".", end="")

# === 5. Connection result ===
if not connected:
    print("\n❌ Could not connect to RoboDK API.")
else:
    print("\n✅ Connected to RoboDK API.")
    print(f"📂 Project loaded: {station.Name()}")


🔁 Closing any existing RoboDK instances...
🚀 Launching RoboDK with project: Exercises1_2.rdk
⏳ Waiting for RoboDK to be ready...
✅ Connected to RoboDK API.
📂 Project loaded: World


We also have to setup the RoboDK API in our Python environment. To do this, we will use the following code:


In [5]:
from robodk.robolink import *    # RoboDK API
from robodk.robomath import *    # Robot toolbox
from math import pi, radians
from numpy import around
import time

RDK = Robolink()

World=RDK.Item("World")
Aframe=RDK.Item("A-Frame")
Bframe=RDK.Item("B-Frame")
myBrick=RDK.Item("myBrick")
myBrick.setVisible(False)

# Define initial pose (position and orientation)
Aframe_init = TxyzRxyz_2_Pose([30, 40, 50, 0, 0, 0])
Bframe_init = Aframe_init
# Set both frames to the same pose relative to the World
Aframe.setParent(World)
Bframe.setParent(World)
Aframe.setPose(Aframe_init)
Bframe.setPose(Bframe_init)

# Set myBrick nested to B-Frame
myBrick.setParent(Bframe)#Do not maintain the actual absolute POSE
myBrick.setVisible(True)
time.sleep(2)


**a) Find the Pose of Bframe respect World if pose of Aframe is (30,40,50,0,0,0)**

To compute the pose of the B-frame with respect to the World frame, we need to construct a **homogeneous transformation matrix (HT)**. This matrix combines both a translation and a rotation, applied in sequence, to describe the pose of an object or coordinate system in 3D space.

According to the diagram, the B-frame is first translated by **(0, 60, 0)** with respect the coordinate system of the B-frame. Then, a **rotation of 45° is applied around the X-axis of the mobile frame (Xb)**.

We represent the transformation from A-frame to B-frame as the product of a translation and a rotation, in this order because the transformations are made with respect to the mobile B-frame:

$$
^{Aframe}Pose_{Bframe} = T(0, 60, 0) \cdot R_x(45^\circ)
$$

Where:

- $T(0, 60, 0)$ is a translation of 60 mm along the Y-axis of the B-frame.

- $R_x(45^\circ)$ is a rotation of 45° around the X-axis of the B-frame.

Translation matrix:
$$
T(0, 60, 0) =
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 60 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

Rotation matrix about the X-axis:
$$
R_x(45^\circ) =
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & \cos(45^\circ) & -\sin(45^\circ) & 0 \\
0 & \sin(45^\circ) & \cos(45^\circ) & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

The Pose of the B-frame with respect to the A-frame can be expressed as:
$$
^{Aframe}Pose_{Bframe} =
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & \cos(45^\circ) & -\sin(45^\circ) & 60 \\
0 & \sin(45^\circ) & \cos(45^\circ) & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

We are given that the pose of the A-frame with respect to the World frame is:

$$
^{World}Pose_{Aframe} =
\begin{bmatrix}
1 & 0 & 0 & 30 \\
0 & 1 & 0 & 40 \\
0 & 0 & 1 & 50 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

This means that the origin of the A-frame is located at coordinates (30, 40, 50), and it is aligned with the World coordinate system (no rotation).

**Final pose of the B-frame with respect to the world**

The final pose of the B-frame is obtained by:

$$
^{World}Pose_{Bframe} = ^{World}Pose_{Aframe} \cdot ^{Aframe}Pose_{Bframe}
$$

This operation results in:

$$
^{World}Pose_{Bframe} =
\begin{bmatrix}
1 & 0 & 0 & 30 \\
0 & \cos(45^\circ) & -\sin(45^\circ) & 100 \\
0 & \sin(45^\circ) & \cos(45^\circ) & 50 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

This final matrix indicates that the B-frame is positioned at **(30, 100, 50)** and has undergone a **rotation of 45° around its local X-axis** relative to the A-frame.

**The position of CG of the brick with respect to the World after the movement is:**

$$
^{World}CG_{after} = ^{World}HT_{Bframe} \cdot ^{Bframe}CG
$$

This operation results in:

$$
^{World}Pose_{after} = (50, 98.2, 62.4, 1)
$$

In [11]:
#Move the object with transformation
ty=60
rx=radians(45)
Bframe_final=Bframe_init*transl(0,ty,0)*rotx(rx)
Bframe.setPose(Bframe_final)
time.sleep(2)
print('Pose B with respect W: '+repr(Bframe_final))

gc_b=[20, 7.5, 10, 1]
gc_w=Bframe_final*gc_b
print('Position CG with respect W: '+repr(around(gc_w, decimals=1)))

Pose B with respect W: Matrix: (4, 4)
Pose(30.000, 100.000, 50.000,  45.000, 0.000, -0.000):
[[ 1.000, 0.000, 0.000, 30.000 ],
 [ 0.000, 0.707, -0.707, 100.000 ],
 [ 0.000, 0.707, 0.707, 50.000 ],
 [ 0, 0.000, 0.000, 1 ]]

Position CG with respect W: array([50. , 98.2, 62.4,  1. ])


---
**b) Find the position of the center of gravity of brick with respect to the Bframe, before and after processing the movement**

Brick dimensions

- Length (X): 40 mm  
- Width (Y): 15 mm  
- Height (Z): 20 mm

The center of gravity of a rectangular solid is located at its geometric center:

$$
^{Bframe}CG =
\left(
\frac{40}{2},\ \frac{15}{2},\ \frac{20}{2}
\right)
=
(20,\ 7.5,\ 10)
$$


**Before the movement**

The brick is initially placed in the B-frame without any offset or rotation. Therefore, the center of gravity with respect to the B-frame is:

$$
^{Bframe}CG_{before} = (20,\ 7.5,\ 10)
$$

**After the movement**

Since the B-frame is moved as a rigid body and the brick is fixed to it, the center of gravity of the brick **does not change with respect to the B-frame**:

$$
^{Bframe}CG_{after} = (20,\ 7.5,\ 10)
$$



---

**c) Find the position of the center of gravity of brick with respect to the A-frame, before and after processing the movement**

From section **b)**, the CG in B-frame coordinates is:

$
^{Bframe}CG = (20,\ 7.5,\ 10)
$

**Before the movement**

Before applying any transformation, the B-frame is aligned with the A-frame at the same position and orientation. Therefore, the CG expressed in the A-frame is exactly the same:

$$
^{Aframe}CG_{before} = ^{Bframe}CG = (20,\ 7.5,\ 10)
$$

**After the movement**

To express the CG with respect to the A-frame **after** the movement, we must apply:

Let:

- $^{Bframe}CG$: coordinates of the center of gravity in B-frame
- $^{Aframe}Pose_{Bframe}$: transformation from B-frame to A-frame (defined in part a)

We apply:

$$
^{Aframe}CG_{after} = ^{Aframe}Pose_{Bframe} \cdot ^{Bframe}CG
$$

Using:

$$
^{Aframe}Pose_{Bframe} =
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & \cos(45^\circ) & -\sin(45^\circ) & 60 \\
0 & \sin(45^\circ) & \cos(45^\circ) & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

and

$$
^{Bframe}CG =
\begin{bmatrix}
20 \\
7.5 \\
10 \\
1
\end{bmatrix}
$$

we compute the product:

$$
^{Aframe}CG_{after} =
\begin{bmatrix}
20 \\
60 + 7.5\cos(45^\circ) - 10\sin(45^\circ) \\
7.5\sin(45^\circ) + 10\cos(45^\circ) \\
1
\end{bmatrix}
$$

By replacing $\cos(45^\circ)$ and $\sin(45^\circ)$ with 0.7071, we get the position of the center of gravity after the movement:


$$
^{Aframe}CG_{after} \approx (20,\ 58.23,\ 12.37, 1)
$$

---

In [13]:
gc_b=[20, 7.5, 10, 1]
gc_a=transl(0,ty,0)*rotx(rx)*gc_b

print('Position GC with respect Aframe before movement: '+repr(gc_b))
print('Position GC with respect Aframe after movement: '+repr(around(gc_a, decimals=1)))


Position GC with respect Aframe before movement: [20, 7.5, 10, 1]
Position GC with respect Aframe after movement: array([20. , 58.2, 12.4,  1. ])


**d) Solve also the three previous questions analytically and using roboDK**

We can generate:
- The complete document in Markdown and save it on a file called "Exemple1_auto.md"
- The complete python code and save it on a file called "Exemple1_auto.py" 


In [24]:
%pip install nbformat

  Using cached importlib_metadata-6.7.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached zipp-3.15.0-py3-none-any.whl.metadata (3.7 kB)
   ---------------------------------------- 77.4/77.4 kB 2.2 MB/s eta 0:00:00
Using cached importlib_metadata-6.7.0-py3-none-any.whl (22 kB)
   ---------------------------------------- 90.4/90.4 kB 5.3 MB/s eta 0:00:00
   ---------------------------------------- 63.0/63.0 kB ? eta 0:00:00
   ---------------------------------------- 62.6/62.6 kB ? eta 0:00:00
Using cached zipp-3.15.0-py3-none-any.whl (6.8 kB)


In [14]:
from nbformat import read
from pathlib import Path

notebook_file = "POSE_Exercise1_student_solution.ipynb"
output_py = "Exercise1_auto.py"

with open(notebook_file, "r", encoding="utf-8") as f:
    nb = read(f, as_version=4)

selected_cells = []
for cell in nb.cells:
    if cell.cell_type == "code":
        tags = cell.metadata.get("tags", [])
        if "export" in tags:
            selected_cells.append(cell.source)

Path(output_py).write_text("\n\n".join(selected_cells), encoding="utf-8")
print(f"✅ Exported {len(selected_cells)} cells to {output_py}")


✅ Exported 3 cells to Exercise1_auto.py


In [26]:
from nbformat import read
from pathlib import Path

# Input notebook
notebook_file = "POSE_Exercise1_student_solution.ipynb"

# Output files
output_py = "Exercise1_auto.py"
output_md = "Exercise1_notes.md"

# Load notebook
with open(notebook_file, "r", encoding="utf-8") as f:
    nb = read(f, as_version=4)

# Prepare cell content
code_cells = []
markdown_cells = []

for cell in nb.cells:
    tags = cell.metadata.get("tags", [])
    if "export" in tags:
        if cell.cell_type == "code":
            code_cells.append(cell.source)
        elif cell.cell_type == "markdown":
            markdown_cells.append(cell.source)

# Save .py file
Path(output_py).write_text("\n\n".join(code_cells), encoding="utf-8")
print(f"✅ Exported {len(code_cells)} code cells to {output_py}")

# Save .md file
Path(output_md).write_text("\n\n---\n\n".join(markdown_cells), encoding="utf-8")
print(f"📝 Exported {len(markdown_cells)} markdown cells to {output_md}")


✅ Exported 3 code cells to Exercise1_auto.py
📝 Exported 6 markdown cells to Exercise1_notes.md


We can run the generated "Exemple1_auto.py" verify we obtain the same results.

<img src="images/ex1_dk.png" width="300"/>


In [24]:
import subprocess
from pathlib import Path
import sys

# Get the current script directory and script path
notebook_dir = Path().resolve()
script_path = (notebook_dir / "Exercise1_auto.py").resolve()

# Run the script and capture its output
result = subprocess.run(
    [sys.executable, str(script_path)],
    capture_output=True,
    text=True
)

# Print the captured output
print("📤 STDOUT:")
print(result.stdout)

print("❌ STDERR (if any):")
print(result.stderr)

📤 STDOUT:
Pose B with respect W: Matrix: (4, 4)
Pose(30.000, 100.000, 50.000,  45.000, 0.000, -0.000):
[[ 1.000, 0.000, 0.000, 30.000 ],
 [ 0.000, 0.707, -0.707, 100.000 ],
 [ 0.000, 0.707, 0.707, 50.000 ],
 [ 0, 0.000, 0.000, 1 ]]

Position CG with respect W: array([50. , 98.2, 62.4,  1. ])
Position GC with respect Aframe before movement: [20, 7.5, 10, 1]
Position GC with respect Aframe after movement: array([20. , 58.2, 12.4,  1. ])

❌ STDERR (if any):

